# 컴퓨팅 대상 사용

스크립트를 Azure Machine Learning 작업으로 실행하는 경우 작업 실행에 대한 실행 컨텍스트를 정의해야 합니다. 한 가지 주요 구성은 스크립트가 실행될 컴퓨팅 대상입니다. 로컬 워크스테이션(이 경우 컴퓨팅 인스턴스) 또는 주문형으로 프로비전되는 Azure Machine Learning 관리형 컴퓨팅 클러스터와 같은 원격 컴퓨팅 대상일 수 있습니다.

이 Notebook에서는 컴퓨팅 클러스터를 만들고 작업에 대한 컴퓨팅 대상을 살펴봅니다.

## 시작하기 전에

이 Notebook에서 코드를 실행하려면  **최신 버전의 azureml-ai-ml** 패키지가 필요합니다. 아래 셀의 명령을 실행하여 이 패키지가 설치되어 있는지 확인합니다.

> **고**:
> **azure-ai-ml** 패키지가 설치되지 않은 경우 를 실행 `pip install azure-ai-ml` 하여 설치합니다.

In [ ]:
## 작업 영역에 연결

필요한 SDK 패키지를 설치했으므로 작업 영역에 연결할 수 있습니다.

작업 영역에 연결하려면 식별자 매개 변수(구독 ID, 리소스 그룹 이름 및 작업 영역 이름)가 필요합니다. 리소스 그룹 이름 및 작업 영역 이름이 이미 채워져 있습니다. 명령을 완료하려면 구독 ID만 필요합니다.

필요한 매개 변수를 찾으려면 Studio의 오른쪽 위에 있는 구독 및 작업 영역 이름을 클릭합니다. 오른쪽에 창이 열립니다.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 구독 ID를 복사하고 **YOUR-SUBSCRIPTION-ID** 를 복사한 값으로 바꿉니다. </p>

## 컴퓨팅 클러스터 만들기

대부분의 경우 로컬 컴퓨팅 리소스만으로는 대량의 데이터를 처리해야 하는 복잡한 실험이나 장기 실행 실험을 처리하기 어려울 수 있습니다. 이러한 상황에서는 클라우드에서 컴퓨팅 리소스를 동적으로 만들고 사용하는 기능을 활용할 수 있습니다. Azure Machine Learning은 작업 영역에서 정의하고 작업을 실행하는 데 사용할 수 있는 다양한 컴퓨팅 대상을 지원합니다. 리소스를 사용할 때만 비용을 지불합니다.

Python SDK 또는 Azure CLI를 사용하여 [Azure Machine Learning 스튜디오](https://ml.azure.com) 컴퓨팅 클러스터를 만들 수 있습니다. 다음 코드 셀은 작업 영역에 컴퓨팅 클러스터 이름이 `aml-cluster`있는지 확인하고, 존재하지 않는 경우 만듭니다.

In [ ]:
컴퓨팅 클러스터를 만든 후에는 다음에 대한 구성만 변경할 수 있습니다.

- `min_instances`: 최소 노드 수
- `max_instances`: 최대 노드 수
- `idle_time_before_scale_down`: 스케일 다운 전 유휴 시간

현재 컴퓨팅 클러스터 `aml-cluster` 는 최대 하나의 노드만 확장할 수 있습니다. 병렬 컴퓨팅을 허용하도록 을 2로 변경해 보겠습니다.

In [ ]:
컴퓨팅 클러스터가 업데이트되면 해당 특성을 인쇄하여 해당 구성을 확인할 수 있습니다.

## 모델을 학습시키는 스크립트 만들기

모델을 학습하려면 먼저 **src** 폴더에 **diabetes_training.py** 스크립트를 만듭니다. 스크립트는 학습 데이터와 동일한 폴더에 있는 **diabetes.csv** 파일을 사용합니다.

In [ ]:
## 컴퓨팅 클러스터에서 작업 실행

이제 만든 컴퓨팅 클러스터에서 작업을 실행할 준비가 되었습니다.

> **고**:
> 컴퓨팅 클러스터의 크기를 0개에서 1개 노드로 조정해야 하므로 작업을 시작하는 데 다소 시간이 걸립니다. 컴퓨팅 클러스터가 준비되면 스크립트가 실행됩니다. 작업이 완료되면 컴퓨팅 클러스터는 0개 노드로 축소됩니다. 컴퓨팅 **페이지에서 컴퓨팅** 클러스터의 상태를 검토할 수 있습니다.

After you've created a compute cluster, you can only change the configuration for:

- `min_instances`: Minimum number of nodes
- `max_instances`: Maximum number of nodes
- `idle_time_before_scale_down`: Idle time before scale down

Currently, your compute cluster `aml-cluster` can only scale do a maximum of one node. Let's change that to two, to allow for parallel compute.

In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="aml-cluster",
    max_instances=2,
)
ml_client.begin_create_or_update(cluster_scale)

When the compute cluster is updated, you can verify its configuration by printing its attributes.

In [ ]:
cpu_cluster = ml_client.compute.get("aml-cluster")

print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

## Create a script to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

## Run a job on a compute cluster

Now, you're ready to run the job on the compute cluster you created.

> **Note**:
> The job will take some time to start as the compute cluster will need to scale from zero to one node. Once the compute cluster is ready, the script will be run. When the job has finished, the compute cluster will scale back down to zero nodes. You can review the compute cluster's status in the **Compute** page.

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)